<a href="https://colab.research.google.com/github/lemil/ai-motorcycle-sme/blob/main/AI_and_motorcycles_test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🏍️ Building with AI - the Ultimate Custom Motorcycle Advisor! 🏍️
Revolutionize your motorcycle building journey with the powerful combination of Llama2, Llama-Index, and LangChain! 🚀 Imagine a cutting-edge AI model that not only talks about custom motorcycles but also provides expert recommendations tailored to your preferences. 🗣️💡 Harness Llama2's efficient indexing system to process vast motorcycle data, while LangChain's language prowess crafts coherent conversations. 🧠🗂️ Whether you're a newbie or a seasoned rider, this AI-powered advisor will guide you through every step, from choosing the perfect parts to styling your dream ride. 🛠️🔧 Say hello to your personal motorcycle guru and take the first step towards building your dream two-wheeled masterpiece!

🏆🔥 #AI #MotorcycleBuilding #CustomBikes #Llama2 #LangChain


Thanks!

Based on the awesome work publised by Ritesh S, check more here https://www.youtube.com/@RitheshSreenivasan

In [ ]:
!pip install pypdf
!pip install python-dotenv

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install llama-index


In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM

In [ ]:
documents = SimpleDirectoryReader("/content/sample_data/").load_data()


In [ ]:
from llama_index.prompts.prompts import SimpleInputPrompt

system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /roo

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)


In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
documents = SimpleDirectoryReader("/content/sample_data/").load_data()

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
query_engine = index.as_query_engine()
#Lets start with tome basic questions
#This topic is widely described in the indexed document
response = query_engine.query("how can I build a build a boober style motorcycle?")
print(response)

Building a bobber-style motorcycle involves several steps, including selecting a donor bike, deciding on a look, sourcing parts, and fabricating and painting the bike. Here's a step-by-step guide to help you get started:
1. Select a Donor Bike: Look for a bike with a simple, unadorned frame and a minimalist engine. Choose a bike that's lightweight and has a low seat height for a more aggressive stance. Consider using a vintage bike or a newer model with a stripped-down configuration.
2. Deciding on a Look: Bobbers are all about simplicity, so keep the design clean and minimalist. Consider a flat black paint job with minimal graphics or decals. You can also add some personal touches, such as a unique seat or custom exhaust.
3. Sourcing Parts: Look for parts that are easy to find and affordable. You'll need a new seat, handlebars, and footpegs. You may also want to replace the exhaust system, brakes, and tires. Consider using aftermarket parts or sourcing them from


In [ ]:
#Now I am asking for specific items present in the indexed document
response = query_engine.query("What brand and model do your recommend to use as Donor Bike?")
print(response)

Based on the information provided, I recommend using a British bike from the 60's or 70's as a donor bike. Specifically, a Triumph or BSA 500 or 650 twin would be a good choice. These bikes have the right lines and angles, but are now fully restored or expensive, making them difficult to find as a donor bike. Alternatively, an Italian bike such as a Ducati non-Desmo 250, 350, or 450 single could also be used as a donor bike. However, Japanese bikes are also a good option, particularly single and twin cylinder models like the Yamaha XS650, Honda CB450, or Suzuki 650 Savage. Ultimately, the choice of donor bike will depend on personal taste and what is available in the local market.


In [ ]:
#Now, I am asking something that the answer is not present in the indexed document.
response = query_engine.query("What indian brand and model do your recommend to use as Donor Bike?")
print(response)

I cannot recommend a specific Indian brand and model as a donor bike without knowing more about your preferences and requirements. However, I can provide some general guidance on factors to consider when choosing a donor bike.
Firstly, it's important to consider the type of bike you want to build. Are you looking for a classic cafe racer, a modern scrambler, or a minimalist bobber? Different bike styles require different donor bikes.
Secondly, think about the engine size and configuration. A larger engine can provide more power and torque for a heavier bike, but it may also be more expensive to maintain and repair. A smaller engine may be more practical and cost-effective, but it may not provide enough power for your desired build.
Thirdly, consider the bike's condition and history. A bike that has been well-maintained and has a clean title is generally a better choice than one that has been neglected or has a questionable history.
Lastly, think about your budget. Donor bikes can range

In [ ]:
#Rephrasing the previous questions
response = query_engine.query("What indian brand and model do your recommend to use as the donor bike for building a cafe racer?. I don not mind about the cost")
print(response)

Based on the context information provided, I would recommend using a Yamaha SR500 as the donor bike for building a cafe racer. The Yamaha SR500 is a classic model that was produced from 1986 to 2008 and is known for its reliability and durability. It has a simple and clean design, which makes it an ideal candidate for a cafe racer build. Additionally, the SR500 has a relatively low price point compared to other classic models, which can help keep costs down during the build process. Overall, the Yamaha SR500 is a great choice for building a cafe racer on a budget.
